In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from transformers import DataCollatorWithPadding
import evaluate
import torch.cuda
import pandas as pd
from sklearn.metrics import classification_report
from nltk.corpus import stopwords

2024-04-30 18:58:10.439457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 18:58:10.439500: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 18:58:10.439518: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-30 18:58:10.445834: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 18:58:11.075512: W tensorflow/compiler/

In [2]:
stop_words = stopwords.words('english')
test_df = pd.read_csv('../data/csv/test.csv')

In [3]:
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

In [4]:
def encode(examples):
    return tokenizer( examples["text"],
                     truncation=True, 
                     padding=True)

In [9]:
test_df['text'] = test_df['text'].str.lower().str.split().apply(lambda x: [item for item in x if item not in stop_words]).apply(lambda x: " ".join(x)).replace('\d+', '', regex=True)
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(encode, batched=True)

Map:   0%|          | 0/2998 [00:00<?, ? examples/s]

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
id2label = {0: "NON_VIOLATED", 1: "VIOLATED"}
label2id = {"NON_VIOLATED": 0, "VIOLATED": 1}
model = AutoModelForSequenceClassification.from_pretrained("../models/ECHR/albert_echr_model/removed_stopwords/accuracy/checkpoint-888", num_labels=2, id2label=id2label, label2id=label2id)
model.to(device)

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [10]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)

In [11]:
results = pipe(tokenizer.decode(case, clean_up_tokenization_spaces=True, skip_special_tokens=True) for case in test_dataset['input_ids'])

In [12]:
predictions = list(result['label'] for result in results)

In [13]:
report = classification_report(list('NON_VIOLATED' if outcome==0 else 'VIOLATED' for outcome in test_dataset['labels']), predictions)

In [14]:
print(report)

              precision    recall  f1-score   support

NON_VIOLATED       0.63      0.68      0.65      1024
    VIOLATED       0.83      0.79      0.81      1974

    accuracy                           0.75      2998
   macro avg       0.73      0.74      0.73      2998
weighted avg       0.76      0.75      0.76      2998

